In [1]:
import pandas as pd
import numpy as np

def find_optimal_transport(C, G, P):
    """
    Find the optimal transport matrix T that maximizes the profit of the transport
    C: Cost matrix (buy price, buy quantity, sell price, sell quantity)
    G: Transportation cost matrix 
    P: Failure probability matrix
    """
    


In [13]:
import numpy as np
from scipy.optimize import linprog
import time

def find_optimal_transport(C, G, P):
    N = C.shape[0]  # Number of cities

    # Constraints
    # Supply constraints for each city
    supply_constraints = np.zeros((N, N * N))
    for i in range(N):
        supply_constraints[i, i*N:(i+1)*N] = 1
    supply_bounds = C[:, 3]

    # Demand constraints for each city
    demand_constraints = np.zeros((N, N * N))
    for j in range(N):
        demand_constraints[j, j::N] = 1
    demand_bounds = C[:, 2]


    # Combine constraints
    constraints = np.concatenate((supply_constraints, demand_constraints))
    
    bounds = np.concatenate((supply_bounds, demand_bounds))

    # Objective: Maximize profits
    # linprog minimizes the objective, so we need to negate the profit matrix
    c = np.zeros(N*N)
    for i in range(N):
        for j in range(N):
            if i != j:
                # Calculate expected profit: selling price - buying price - transportation cost adjusted for closure probability
                c[i*N+j] = -(C[j][1] - C[i][0] - G[i][j]) * (1 - P[i][j])

    # Bounds for each variable should be between 0 and the minimum of supply/demand for its route
    variable_bounds = [(0, None) for _ in range(N * N)]
    # Solve the linear programming problem
    res = linprog(c, A_ub=constraints, b_ub=bounds, bounds=variable_bounds, method='highs')

    # Extract the transportation plan from the solution
    # The solution vector is flattened, so we need to reshape it back to N x N
    transportation_plan = res.x.reshape(N, N)
    # print(transportation_plan)

    # Create the instructions array
    instructions = []
    for i in range(N):
        for j in range(N):
            if transportation_plan[i][j] > 0:
                instructions.append([i, j, transportation_plan[i][j]])

    # Convert to numpy array and return
    return np.array(instructions)

#Example usage:
C = np.array([[98,100,100,100],
[98,100,100,100],
[102,103,100,100],
[102,103,100,100],
[102,103,100,100]])
G = np.array([[0,1,1,1,1],
[1,0,1,1,1],
[1,1,0,1,1],
[1,1,1,0,1],
[1,1,1,1,0]])
P = np.array([[0,0,0.5,0,0],
[0,0,0,0,0],
[0.5,0,0,0,0],
[0,0,0,0,0],
[0,0,0,0,0]])

# Get the transportation plan
start = time.time()
plan = create_transportation_plan(C, G, P)
stop = time.time()
print(plan)
print(stop - start)

# print memory usage
import os
import psutil
process = psutil.Process(os.getpid())
print(process.memory_info().rss / 1024 ** 2)

[[  0.   4. 100.]
 [  1.   3. 100.]]
0.0018186569213867188
71.79296875


In [10]:
# generate two random matrices
C = np.random.randint(100, size=(4, 4))
G = np.random.randint(100, size=(4, 4))

print(C)
print(G)

T = np.concatenate((C, G), axis=1)
del C
del G
print(T)


[[99 96 91 26]
 [15 95 29  7]
 [75 45 40 22]
 [65 33 53 17]]
[[33 71 59 77]
 [87  1 97 94]
 [67 52  9 17]
 [42 94 78 56]]
[[99 96 91 26 33 71 59 77]
 [15 95 29  7 87  1 97 94]
 [75 45 40 22 67 52  9 17]
 [65 33 53 17 42 94 78 56]]


In [14]:
C = np.array(
    [
        [98, 100, 100, 100],
        [98, 100, 100, 100],
        [102, 103, 30, 100],
        [102, 103, 50, 100],
        [101, 103, 50, 100],
    ]
)

G = np.array(
    [
        [0, 1, 1, 1, 1],
        [1, 0, 1, 1, 1],
        [1, 1, 0, 1, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 1, 0],
    ]
)

P = np.array(
    [
        [0, 0.2, 0.5, 0, 0.5],
        [0.2, 0, 0.5, 0.3, 0],
        [0.5, 0.5, 0, 0, 0.3],
        [0, 0.3, 0, 0, 0],
        [0.5, 0, 0.3, 0, 0],
    ]
)